In [1]:
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\programdata\anaconda3\lib\site-packages (0.0)


In [2]:
import pandas as pd
from datetime import datetime as dt
import numpy as np
import requests
import config
import json
import time
import datetime
from pprint import pprint

In [3]:
#pull in the postCovid player data
df = pd.read_csv('data_files/player_stats_2019_pst.csv')

In [4]:
df

,Unnamed: 0,First_Name,Last_Name,player_id,Position,Points,Free_Throw_Percent,Two_Pt_Percent,Three_Pt_Percent,Assists,...,Turnovers,Team,Home_Away,win,Team_points,Min_played,Crowd,Stadium_Cap,game_id,game_date
0,0,Torrey,Craig,c14eba17-879a-461e-9413-bf71a378de6e,G,11,0.0,50.0,60.0,0,...,1,Nuggets,1,1,135,21:25,Covid,1200,6541eb3b-eb31-4f6f-a662-94e9d2640c99,2020-08-17T17:30:00+00:00
1,1,Vlatko,Cancar,957e63a8-d87d-47d2-89cc-416afa292d80,F,0,0.0,0.0,0.0,0,...,0,Nuggets,1,1,135,00:00,Covid,1200,6541eb3b-eb31-4f6f-a662-94e9d2640c99,2020-08-17T17:30:00+00:00
2,2,Mason,Plumlee,9570a938-324f-40e8-92dd-8a4fcf4a953b,F-C,2,0.0,100.0,0.0,2,...,1,Nuggets,1,1,135,10:50,Covid,1200,6541eb3b-eb31-4f6f-a662-94e9d2640c99,2020-08-17T17:30:00+00:00
3,3,Jerami,Grant,5b315e15-6633-4ce2-8200-71b821553314,F,19,87.5,50.0,40.0,3,...,0,Nuggets,1,1,135,39:56,Covid,1200,6541eb3b-eb31-4f6f-a662-94e9d2640c99,2020-08-17T17:30:00+00:00
4,4,Troy,Daniels,562677c1-2a13-4934-a801-a6c88c619e83,G,0,0.0,0.0,0.0,0,...,0,Nuggets,1,1,135,00:00,Covid,1200,6541eb3b-eb31-4f6f-a662-94e9d2640c99,2020-08-17T17:30:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2692,2692,Anthony,Davis,ea8826b8-1f76-4eab-b61e-ffcb176880f3,C,19,71.4,50.0,0.0,3,...,3,Lakers,0,1,106,35:06,Covid,8000,b56b052b-b341-46c0-916d-16982d80961d,2020-10-11T23:30:00+00:00
2693,2693,Alex,Caruso,8936c15e-f9d5-4b9a-80b9-8f6b7dcf7a17,G,4,0.0,40.0,0.0,5,...,1,Lakers,0,1,106,32:42,Covid,8000,b56b052b-b341-46c0-916d-16982d80961d,2020-10-11T23:30:00+00:00
2694,2694,JaVale,McGee,34ff5b55-42ef-40c0-a2c4-9c28b5476a03,C-F,0,0.0,0.0,0.0,0,...,0,Lakers,0,1,106,00:00,Covid,8000,b56b052b-b341-46c0-916d-16982d80961d,2020-10-11T23:30:00+00:00
2695,2695,Kentavious,Caldwell-Pope,cecc054e-b9ec-4c71-8bbb-5ee665a8c3a9,G,17,100.0,66.7,28.6,0,...,2,Lakers,0,1,106,33:26,Covid,8000,b56b052b-b341-46c0-916d-16982d80961d,2020-10-11T23:30:00+00:00


In [5]:
#dropping unwanted fields to match preCovid columns
#don't drop Covid games obviously
df_dropped = df[df['Min_played'] != "00:00"]
df_dropped= df_dropped[["Points", "Free_Throw_Percent",
                  "Two_Pt_Percent",
                  "Three_Pt_Percent", "Assists",
                  "Rebounds", "Offensive_Rebounds",
                  "Steals", "Personal_Fouls",
                  "Flagrant_Fouls", "Tech_Fouls",
                  "Turnovers",
                  "Home_Away", "win"
                  ]].reset_index(drop = True)
df_dropped

,Points,Free_Throw_Percent,Two_Pt_Percent,Three_Pt_Percent,Assists,Rebounds,Offensive_Rebounds,Steals,Personal_Fouls,Flagrant_Fouls,Tech_Fouls,Turnovers,Home_Away,win
0,11,0.0,50.0,60.0,0,4,2,0,6,0,0,1,1,1
1,2,0.0,100.0,0.0,2,6,1,1,3,0,0,1,1,1
2,19,87.5,50.0,40.0,3,0,0,1,2,0,0,0,1,1
3,14,0.0,66.7,66.7,4,3,0,1,1,0,0,1,1,1
4,13,0.0,33.3,42.9,1,8,1,0,3,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1684,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,1
1685,19,71.4,50.0,0.0,3,15,4,1,4,0,0,3,0,1
1686,4,0.0,40.0,0.0,5,3,1,1,2,0,0,1,0,1
1687,17,100.0,66.7,28.6,0,2,1,1,5,0,0,2,0,1


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [7]:
X = df_dropped.drop('win', axis=1)
print(X.shape)

(1689, 13)


In [8]:
y = df_dropped['win']
print(y.shape)

(1689,)


In [9]:
X_scaler = MinMaxScaler().fit(X)
X_predict_scaled = X_scaler.transform(X)


In [10]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y_actual = label_encoder.transform(y)

In [11]:
y_actual_categorical = to_categorical(encoded_y_actual)

In [12]:
#load the model
from keras.models import load_model
model = load_model('models/deepLearningSequential.h5')

In [13]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 100)               1400      
_________________________________________________________________
dense_10 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_11 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_12 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_13 (Dense)             (None, 2)                 202       
Total params: 31,902
Trainable params: 31,902
Non-trainable params: 0
_________________________________________________________________


In [14]:
encoded_predictions = model.predict_classes(X_predict_scaled)
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [15]:
print(f"First 10 Predictions:   {prediction_labels[:10]}")
print(f"First 10 Actual labels: {encoded_y_actual[:10].tolist()}")

First 10 Predictions:   [0 1 1 1 1 0 1 1 1 1]
First 10 Actual labels: [1, 1, 1, 1, 1, 1, 1, 1, 1, 0]


In [16]:
pd.DataFrame({"Prediction": prediction_labels, "Actual": encoded_y_actual}).reset_index(drop=True)

,Prediction,Actual
0,0,1
1,1,1
2,1,1
3,1,1
4,1,1
...,...,...
1684,0,1
1685,1,1
1686,0,1
1687,0,1


In [17]:
model_loss, model_accuracy = model.evaluate(
    X_predict_scaled, y_actual_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

53/53 - 0s - loss: 2.0330 - accuracy: 0.5317
Normal Neural Network - Loss: 2.0329573154449463, Accuracy: 0.5316755771636963


In [18]:
score = model.evaluate(X_predict_scaled, y_actual_categorical, verbose=2)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

53/53 - 0s - loss: 2.0330 - accuracy: 0.5317
accuracy: 53.17%
